# Tutorial 1: Tensor Contractions 
source: https://www.tensors.net/tutorial-1

- Initialization of tensors

- Diagrammatic notation for tensors and tensor networks

- Manipulation of tensors via 'permute' and 'reshape' functions

- Binary tensor contractions and computational costs

- Use of 'ncon' routine to contract networks


## T1.1: Diagrammatic notation

For our present purpose, tensors are simply multi-dimensional arrays of (real or complex) numbers. Below are some useful ways to initialize tensors:

In [ ]:
##### Lets initialize some tensors in Julia
using LinearAlgebra
# tensor with randomly generated entries, order 3, dims: 2-by-3-by-4
A = rand(2,3,4)

# identity matrix, order 2, dims: 5-by-5 (New syntax in Julia 0.7+)
B = Matrix{Float64}(I,5,5)

# tensor of 1's, order 4, dims: 2-by-4-by-2-by-4
C = ones(2,4,2,4)

# matrix of 0's, order 2, dims: 3-by-5
D = zeros(3,5)

# initialize complex random tensor
E = rand(2,3,4) + im*rand(2,3,4)

It is convenient to represent tensor networks using a **diagrammatic notation**, where individual tensors are represented as a solid shape with a number of 'legs' that corresponds to the rank of the tensor. Each leg is here labelled with a dummy index (usually a Latin letter: i, j, k, l…) necessary to relate the equation to the diagram. Some examples are presented below.

<img  src="./Figs/Fig.1.1(a).png"  width="600"  align="center" />

The diagrammatic tensor notation is useful for describing networks comprised of multiple tensors. An index shared by two tensors denotes a contraction (or summation) over this index. Examples:

<img  src="./Figs/Fig.1.1(b,c).png"  width="600"  align="center" />


Notice that example in Fig.1.1(b) is equivalent to a matrix multiplication between matrices **A** and **B**, while Fig.1.1(c) produces a rank-3 tensor **D** via the contraction of a network with three tensors. Even in this relatively the simple example, we see that the diagrammatic notation is already easier to interpret than the corresponding index equation. In practice, once we have established a convention for index ordering, we can omit labeling the diagrams with dummy indices which further enhances their clarity.

## T1.2: <font color=red>Permute</font> and <font color=red>reshape</font> operations

<img  src="./Figs/Fig.1.2.png"  width="600"  align="center" />

In [ ]:
##### Ex.1.2(a):Permute
A = rand(4,4,4,4)
Atilda = permutedims(A,[4,1,2,3])

In [ ]:
##### Ex.1.2(b):Reshape
B = rand(4,4,4)
Btilda = reshape(B,4,4^2)

### Technical notes:

- The tensor reshape behaves differently in MATLAB/Julia versus Python due to a difference in convention. Both MATLAB and Julia use column-major order for storing matrices and tensors, such that a d-by-d matrix Bij is stored as a length d^2 vector vk, with k = i + (j-1)×d. In contrast, Python uses row-major order such that a d-by-d matrix Bij is stored as a vector vk, with k = i×d + j. Fortunately this difference in convention does not often have significant consequences in terms of writing tensor network codes, since the choice of convention is not so important so long as it is consistently applied.

- The permute function reorders the storage of the elements of a tensor in computer memory, thus incurs some (often non-negligible) computational cost. In contrast, the reshape function leaves the elements of a tensor unchanged in memory, instead only changing the metadata for how the tensor is to be interpreted (and thus incurs negligible cost).

## T1.3: Binary tensor contractions

The usefulness of permute and reshape functions is that they allow a contraction between a pair of tensors (which we call a binary tensor contraction) to be recast as a matrix multiplication. Although the computational cost (measured in number of scalar multiplications) is the same both ways, it is usually preferable to recast as multiplication as modern hardware performs vectorized operations much faster than when using the equivalent FOR loop. The steps for doing this are outlined below:

<img  src="./Figs/Fig.1.3.png"  width="600"  align="center" />

In [14]:
##### Ex.1.3(a): Binary Tensor Contraction
d = 10;
A = rand(d,d,d,d);  B = rand(d,d,d,d);
Ap  = permutedims(A,[1,3,2,4]);  Bp  = permutedims(B,[1,4,2,3]);
App = reshape(Ap,d^2,d^2);       Bpp = reshape(Bp,d^2,d^2);
Cpp = App*Bpp;
C   = reshape(Cpp,d,d,d,d);

## T1.4: Contraction costs

The computational cost of multiplying a d1-by-d2 dimensional matrix **A** with a d2-by-d3 dimensional matrix **B** is: cost(A×B) = d1∙d2∙d3. Given the equivalence with matrix multiplication, this is also the cost of a binary tensor contraction (where each dimension d1, d2, d3 may now result as the product of several tensor indices from the reshapes).

Another way of computing the cost of contracting **A** and **B** is to take the product of the total dimensions, denoted |dim(**A**)| and |dim(**B**)|, of each tensor divided by the total dimension of the contracted indices, denoted |dim(**A**∩**B**)|. Examples are given below:

<img  src="./Figs/Fig.1.4.png"  width="600"  align="center" />

Broadly speaking, there are two approaches that could be taken to contract a network containing N>2 tensors: (i) in a single step as a direct summation over all internal indices of the network or (ii) as a sequence of N-1 binary contractions. In practice we prefer the latter option, which is either computationally cheaper or an equivalent cost as the former option. Examples:

<img  src="./Figs/Fig.1.4(c).png"  width="600"  align="center" />

Fig.1.4(c), which represents a product of three matrices, illustrates that it is more efficient (in terms of the total number of scalar multiplications) to evaluate the network as a sequence of binary contractions than as a single summation over all internal indices.

In [19]:
##### Ex.1.4(c): Tensor network evaluation
d = 10; A = rand(d,d);  B = rand(d,d); C = rand(d,d);
# Evaluare network via summation over internal indices
F0 = zeros(d,d);
for id = 1:d
    for jd = 1:d
        for kd = 1:d
            for ld = 1:d
                F0[id,jd] = F0[id,jd] + A[id,kd]*B[kd,ld]*C[ld,jd];
            end
        end
    end
end
# Evaluare network via sequence of binary contractions
F1 = (A*B)*C;

<img  src="./Figs/Fig.1.4(d).png"  width="600"  align="center" />


Fig.1.4(d) illustrates that the total cost of contracting a tensor network can depend on the sequence of binary contractions used; here the optimal sequence depends on whether D is larger than d.

## T1.5: Contraction of tensor networks


Given a tensor network composed of N tensors, there are two distinct steps needed to contract the network efficiently:

- determine the optimal sequence of the (N-1) binary tensor contractions,

- evaluate each of the binary contractions in turn as a matrix multiplication by taking the proper tensor permutes and reshapes.


### Notes: determining the optimal contraction sequence:

Usually we refer to the ‘optimal’ sequence at that which minimizes the number of scalar multiplications, but one could also seek to minimize the size of intermediate tensors used in the contraction (if the calculation was memory limited). Often, though not always, these two criteria will coincide.

Given a tensor network with only a few tensors it is often easy to find the optimal sequence ‘manually’ through inspection of the network. For more complicated networks with a large number of tensors it may be necessary to employ an automated search algorithm such as this.

<img  src="./Figs/Fig.1.5(a).png"  width="600"  align="center" />

Once the optimal contraction sequence has been determined, a network can be evaluated by implementing each of the binary contractions in turn. However, using ‘reshape’ and ‘permute’ commands for each binary tensor contraction, although computationally effective, has two significant drawbacks: 
- (i) it results in lengthy code that is error prone and difficult to check
- (ii) it does not allow for the contraction sequence to be easily changed (as, in general, the entire code for the contraction would need to be rewritten to accommodate a different ordering).

### Network contractor ‘ncon’:

The ‘ncon’ function is a useful tool to lessen the programming effort required to implement a tensor network contraction. This function works by automatically performing a desired sequence of permutes, reshapes and matrix multiplications required to evaluate a tensor network. The ‘ncon’ code and detailed instructions for its usage can be found here, or alternatively the code is also presented on the example code page. The first step in using ‘ncon’ to evaluate a network is to make a labelled diagram of the network such that:​

- Each internal index is labelled with a unique positive integer (typically sequential integers starting from 1, although this is not necessary).

- External indices of the diagram (if there are any) are labelled with sequential negative integers [-1,-2,-3,…] which denote the desired index order on the final tensor (with -1 as the first index, -2 as the second etc).

​Following this, the ‘ncon’ routine is called as follows,

 <font color=red> **OutputTensor = ncon(TensorArray, IndexArray, ContOrder),** </font>

 with input arguments defined:

- **TensorArray**: 1D cell array containing the tensors comprising the network

- **IndexArray**: 1D cell array of vectors, where the kth element is a vector of the integer labels from the diagram on the kth tensor from ‘TensorArray’ (ordered following the corresponding index order on this tensor).

- **ContOrder**: a vector containing the positive integer labels from the diagram, used to specify order in which ‘ncon’ contracts the indices. Note that ‘ContOrder’ is an optional input that can be omitted if desired, in which case ‘ncon’ will contract in ascending order of index labels.

<img  src="./Figs/Fig.1.5(b).png"  width="600"  align="center" />

In [28]:
##### Ex.1.5(b): Contraction using ncon
include("ncon.jl")
d = 10;
A = rand(d,d,d); B = rand(d,d,d,d);
C = rand(d,d,d); D = rand(d,d);

TensorArray = Any[A,B,C,D];
IndexArray = Any[[1,-2,2],[-1,1,3,4],[5,3,2],[4,5]];

E = ncon(TensorArray,IndexArray;con_order = [5,3,4,1,2]);


**Notes on ncon**

- If a pair of tensors is connected via multiple indices then 'ncon' will perform the contraction as a single multiplication (as opposed to contracting each index sequentially).

- Can be used to evaluate partial traces (see example below).

- Can be used to combine disjoint tensors into a single tensor (see example below). 
- 
<img  src="./Figs/Fig.1.5(c,d).png"  width="600"  align="center" />

In [ ]:
##### Ex.1.5(c): Partial trace
d = 10;
A = rand(d,d,d,d,d,d);
B = ncon(Any[A],Any[[-1,-2,1,-3,-4,1]]);

In [ ]:
##### Ex.1.5(d): Disjoint networks
d = 10;
A = rand(d,d);
B = rand(d,d);
C = ncon(Any[A,B],Any[[-1,-2],[-3,-4]]);

## Problem Set 1:

<img  src="./Figs/Pb.1(a).png"  width="600"  align="center" />

**Pb.1(b)**

Initialize rank-3 random tensors A, B, C (assuming all indices are dimension d = 20). Write code to evaluate the network contraction (using the specified index orders) in three different ways: 

- As a single summation ​​over all internal indices using FOR loops.

- As a sequence of binary contractions implemented using 'permute' and 'reshape'.

- Using the 'ncon' routine.

Check that all three approaches produce the same output tensor D, and compare their respective computation times. 

<img  src="./Figs/Pb.1(b).png"  width="600"  align="center" />

In [12]:
##### Pb.1(b) - Solutions
using LinearAlgebra
using ITensors
using BenchmarkTools
include("ncon.jl")

d = 20;
A = rand(d,d,d); B = rand(d,d,d); C = rand(d,d,d);
i,j,k,l,m,n = Index(d),Index(d),Index(d),Index(d),Index(d),Index(d)
IA = ITensor(i,j,k); IB = ITensor(j,l,m); IC = ITensor(k,m,n);
for a in 1:d, b in 1:d, c in 1:d
    IA[i=>a,j=>b,k=>c] = A[a,b,c]
    IB[j=>a,l=>b,m=>c] = B[a,b,c]
    IC[k=>a,m=>b,n=>c] = C[a,b,c]
end
##### Evaluate network via index summation
function tempfunct(A,B,C,d)
    D0 = zeros(d,d,d);
    for b1 = 1:d
        for a2 = 1:d
            for c3 = 1:d
                for a1 = 1:d
                    for a3 = 1:d
                        for c1 = 1:d
                            D0[b1,a2,c3] = D0[b1,a2,c3]+A[a1,a2,a3]*B[b1,a1,c1]*C[c1,a3,c3];
                        end
                    end
                end
            end
        end
    end
    return D0
end

t_sum = @elapsed D0 = tempfunct(A,B,C,d);

##### Evaluate network using reshape and permute
function tempfunct2(A,B,C,d)
    Xmid = reshape(reshape(permutedims(B,[1,3,2]),d^2,d)*reshape(A,d,d^2),d,d,d,d);
    D1 = reshape(reshape(permutedims(Xmid,[1,3,2,4]),d^2,d^2)*reshape(C,d^2,d),d,d,d);
    return D1
end

##### Evaluate network using ITensors
function tempfunct3(A,B,C)
    D2 = A*B*C
    return D2
end

t_res = @elapsed D1 = tempfunct2(A,B,C,d)

##### Evaluate using ncon
t_ncon = @elapsed D2 = ncon(Any[A,B,C],Any[[1,-2,2],[-1,1,3],[3,2,-3]]; con_order = [1,2,3], check_network=true);

t_iten = @elapsed D3 = tempfunct3(IA, IB, IC)
##### Compare
tdiffs = [maximum(abs.(D0[:]-D1[:])),maximum(abs.(D1[:]-D2[:])),maximum(abs.(D2[:]-D0[:]))]
ttimes = [t_sum, t_res, t_ncon, t_iten]

t_res =  @elapsed D1 = tempfunct2(A,B,C,d)

##### Evaluate using ncon
t_ncon = @elapsed D2 = ncon(Any[A,B,C],Any[[1,-2,2],[-1,1,3],[3,2,-3]]; con_order = [1,2,3], check_network=true);

t_iten = @elapsed D3 = tempfunct3(IA, IB, IC)
##### Compare
tdiffs = [maximum(abs.(D0[:]-D1[:])),maximum(abs.(D1[:]-D2[:])),maximum(abs.(D2[:]-D0[:]))]
ttimes = [t_sum, t_res, t_ncon]

In [ ]:
@benchmark tempfunct(A,B,C,d)

In [ ]:
@benchmark tempfunct2(A,B,C,d)

In [ ]:
@benchmark ncon(Any[A,B,C],Any[[1,-2,2],[-1,1,3],[3,2,-3]]; con_order = [1,2,3], check_network=true)

In [ ]:
@benchmark  tempfunct3(IA, IB, IC)

In [5]:
using LinearAlgebra
using ITensors
using BenchmarkTools
include("ncon.jl")

d1 = 1000;
d2 = 4;
A = rand(d1,d2,d1); B = rand(d1,d2,d1);
i,j,k,l,m= Index(d1),Index(d2),Index(d1),Index(d2),Index(d1)
IA = ITensor(i,j,k); IB = ITensor(k,l,m);
for a in 1:d1, b in 1:d2, c in 1:d1
    IA[i=>a,j=>b,k=>c] = A[a,b,c]
    IB[k=>a,l=>b,m=>c] = B[a,b,c]
end


function temp1(A,B,d1,d2)
    D1 = reshape(reshape(A,d1*d2,d1)*reshape(B,d1,d2*d1),d1,d2,d2,d1);
    return D1
end


temp1 (generic function with 1 method)

In [6]:
@benchmark temp1(A,B,d1,d2)

BenchmarkTools.Trial: 25 samples with 1 evaluation.
 Range (min … max):  192.209 ms … 207.597 ms  ┊ GC (min … max): 0.15% … 0.89%
 Time  (median):     203.918 ms               ┊ GC (median):    0.91%
 Time  (mean ± σ):   203.175 ms ±   3.411 ms  ┊ GC (mean ± σ):  0.87% ± 0.19%

                                        ▃       ▃ ▃  █ █         
  ▇▁▁▁▁▁▁▁▁▁▁▁▁▁▇▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▇▇▇▇▁█▁▁▇▇▁▇▇█▁█▁▁█▁█▁▁▁▇▁▇▇ ▁
  192 ms           Histogram: frequency by time          208 ms <

 Memory estimate: 122.07 MiB, allocs estimate: 8.

In [7]:
@benchmark IA*IB

BenchmarkTools.Trial: 24 samples with 1 evaluation.
 Range (min … max):  205.131 ms … 260.804 ms  ┊ GC (min … max): 0.34% … 1.69%
 Time  (median):     211.121 ms               ┊ GC (median):    0.87%
 Time  (mean ± σ):   213.398 ms ±  10.340 ms  ┊ GC (mean ± σ):  0.90% ± 0.33%

        █▃                                                       
  ▄▁▁▁▅▅██▁▁▄▄▄▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▄ ▁
  205 ms           Histogram: frequency by time          261 ms <

 Memory estimate: 122.08 MiB, allocs estimate: 33.

In [8]:
@benchmark ncon(Any[A,B],Any[[-1,-2,1],[1,-3,-4]]; con_order = [1], check_network=true)

BenchmarkTools.Trial: 21 samples with 1 evaluation.
 Range (min … max):  234.535 ms … 259.134 ms  ┊ GC (min … max): 1.22% … 2.04%
 Time  (median):     239.756 ms               ┊ GC (median):    1.28%
 Time  (mean ± σ):   242.819 ms ±   7.561 ms  ┊ GC (mean ± σ):  1.61% ± 0.59%

  ▁▁ ▁▁█ ▁▁▁  █ ▁▁      ▁      ▁     ▁▁         ▁ ▁  ▁        ▁  
  ██▁███▁███▁▁█▁██▁▁▁▁▁▁█▁▁▁▁▁▁█▁▁▁▁▁██▁▁▁▁▁▁▁▁▁█▁█▁▁█▁▁▁▁▁▁▁▁█ ▁
  235 ms           Histogram: frequency by time          259 ms <

 Memory estimate: 305.19 MiB, allocs estimate: 214.